## Get Jira Data

In [22]:
from jira import JIRA, JIRAError
from collections import Counter, defaultdict
from datetime import datetime
from time import sleep

import numpy as np
import pandas as pd
import networkx as nx
import plotly.express as px

import os

url = "https://swymcorp.atlassian.net"
email = "ram.vellanki@swymcorp.com"
authtoken = "ENTER JIRA API TOKEN HERE"
filename = 'data_jira.csv'
jira = JIRA(options={'server': url}, basic_auth=(email, authtoken))

try:
    jql = "project=LE AND 'Category[Dropdown]' not in ('v3 Migration', 'Import')AND created >= '-365d'"
    #search issues
    block_size=100
    block_num=0
    jira_search = jira.search_issues(jql, startAt=block_num*block_size, maxResults=block_size,
                                    fields="issuetype, priority, resolution, created, customfield_10079, customfield_10055, customfield_10048, customfield_10045, customfield_10078, customfield_10046, customfield_10049, resolutiondate, reporter, assignee, status")

    if os.path.exists(filename):
        os.remove(filename)
        print("file deleted")
    else:
        print("file doesn't exists so will be created")

    
    #define parameters for writing data
    index_beg = 0
    header = True
    mode = 'w'
    while bool(jira_search):
        data_jira = []
        
        for issue in jira_search:
            issue_key = issue.key
            request_type = str(issue.fields.issuetype)
            
            datetime_creation = issue.fields.created
            if datetime_creation is not None:
                datetime_creation = datetime.strptime(datetime_creation[:19], "%Y-%m-%dT%H:%M:%S")
            
            datetime_resolution = issue.fields.resolutiondate
            if datetime_resolution is not None:
                datetime_resolution = datetime.strptime(datetime_resolution[:19], "%Y-%m-%dT%H:%M:%S")
            
            reporter_login = None
            reporter_name = None
            reporter = issue.raw['fields'].get('reporter', None)
            if reporter is not None:
                reporter_login = reporter.get('key', None)
                reporter_name = reporter.get('displayName', None)
            
            assignee_login = None
            assignee_name = None
            assignee = issue.raw['fields'].get('assignee', None)
            if assignee is not None:
                assignee_login = assignee.get('key', None)
                assignee_name = assignee.get('displayName', None)
                
            status = None
            st = issue.fields.status
            if st is not None:
                status = st.name
                
            priority = None
            pty = issue.fields.priority if hasattr(issue.fields, "priority") else None
            if pty is not None:
                priority = pty
            
            resolution = None
            resn = issue.fields.resolution if hasattr(issue.fields, "resolution") else "NA"
            if resn is not None:
                resolution = resn
            else:
                resolution = "NA"
             
            platformplan = None
            pp = issue.fields.customfield_10048 if hasattr(issue.fields, "customfield_10048") else "NA"
            if pp is not None:
                platformplan = pp.value
            else:
                platformplan = "NA"

            
            swymplan = None
            sp = issue.fields.customfield_10049 if hasattr(issue.fields, "customfield_10049") else "NA"
            if sp is not None:
                swymplan = sp.value
            else:
                swymplan = "NA"
            
            shopurl = None
            surl = issue.fields.customfield_10045 if hasattr(issue.fields, "customfield_10045") else "NA"
            if surl is not None:
                shopurl = surl
            else:
                shopurl = "NA"
            
            platform = None
            pform = issue.fields.customfield_10046 if hasattr(issue.fields, "customfield_10046") else "NA"
            if pform is not None:
                platform = pform
            else:
                platform = "NA"
            
            teamname = None
            tname = issue.fields.customfield_10078 if hasattr(issue.fields, "customfield_10078") else "Unassigned"
            if tname is not None:
                teamname = tname.value
            else:
                teamname = "Unassigned"
            
            category = None
            cat = issue.fields.customfield_10055 if hasattr(issue.fields, "customfield_10055") else "NA"
            if cat is not None:
                category = cat.value
            else:
                category = "NA"
                
            #customfield_10079
            rca = None
            rcause = issue.fields.customfield_10079 if hasattr(issue.fields, "customfield_10079") else "TBD"
            if rcause is not None:
                rca = rcause.value
            else:
                rca = "TBD"
                
            data_jira.append((issue_key, priority, resolution, category, rca, teamname, platformplan, swymplan, shopurl, platform, 
                              request_type, datetime_creation, datetime_resolution, reporter_login, reporter_name, 
                              assignee_login, assignee_name, status))
            
        index_end = index_beg + len(data_jira)
        data_jira = pd.DataFrame(data_jira, index=range(index_beg, index_end),
                                columns = ['Issue Key', 'Priority', 'Resolution','Category', 'Root Cause', 'TeamName', 'Platform Plan','Swym Plan',
                                           'ShopURL','Platform','Request Type', 'Datetime creation', 
                                           'Datetime Resolution', 'Reporter Login', 'Reporter Name', 
                                           'Assignee Login', 'Assignee Name', 'Status'])
        
        data_jira.to_csv(path_or_buf=filename, sep=',', header=header, index=True, index_label = 'N', mode=mode)
    
        block_num = block_num + 1
        index_beg = index_end
        header = False
        mode = 'a'

        #print issues read
        if block_num % 50 == 0:
            print(block_num * block_size)

        sleep(1) #reduce load on system
        
        jira_search = jira.search_issues(jql, startAt=block_num*block_size, maxResults=block_size,
                                    fields="issuetype, priority, resolution, created, customfield_10079, customfield_10055, customfield_10048, customfield_10045, customfield_10078, customfield_10046, customfield_10049, resolutiondate, reporter, assignee, status")
        
    jira.close()

except JIRAError as e:
    jira.close()
    print('Error', e)
except AttributeError as x:
    print(x)


file deleted


## Weekly Data

In [23]:
# Load Data
import pandas as pd
datadf = pd.read_csv(filename, sep=',', header =0, encoding='utf_8_sig')
# Set week parameters
weekstartdt = "2022-02-21"
weekenddt = "2022-02-27"
#total rows
len(datadf)

824

In [34]:
from datetime import datetime as dt
datadf['CreateDateM'] = datadf['Datetime creation'].astype('datetime64[M]')

datadf['CreateDate'] = datadf['Datetime creation'].astype('datetime64[D]')
datadf['ResolvedDate'] = datadf['Datetime Resolution'].astype('datetime64[D]')

after_start_date = datadf["CreateDate"] >= dt.strptime(weekstartdt, "%Y-%m-%d")
before_end_date = datadf["CreateDate"] < dt.strptime(weekenddt, "%Y-%m-%d") 

between_two_dates = after_start_date & before_end_date

res_after_start_date = datadf["ResolvedDate"] >= dt.strptime(weekstartdt, "%Y-%m-%d")
res_before_end_date = datadf["ResolvedDate"] < dt.strptime(weekenddt, "%Y-%m-%d") 
res_between_two_dates = res_after_start_date & res_before_end_date


lastweekdfcreated = datadf.loc[between_two_dates]
lastweekdfresolved = datadf.loc[res_between_two_dates]

lastweekdf.head(5)

,N,Issue Key,Priority,Resolution,Category,Root Cause,TeamName,Platform Plan,Swym Plan,ShopURL,...,Datetime creation,Datetime Resolution,Reporter Login,Reporter Name,Assignee Login,Assignee Name,Status,CreateDate,ResolvedDate,CreateDateM
4,4,LE-1525,Highest,NaN,Problem/Bug,TBD,Unassigned,Shopify Plus,Enterprise,mv-row.myshopify.com,...,2022-02-25 10:33:23,NaN,NaN,Gopinath Nagalingam,NaN,Nilarjun Das,Waiting On Customer,2022-02-25,NaT,2022-02-01
5,5,LE-1524,Medium,NaN,Customization,TBD,Apps,Shopify Professional,Starter,maison-mayle-store.myshopify.com,...,2022-02-25 02:59:07,NaN,NaN,Anshu Bhardwaj,NaN,Nilarjun Das,Open,2022-02-25,NaT,2022-02-01
6,6,LE-1523,Medium,NaN,Problem/Bug,TBD,Platform,Shopify Professional,Starter,marigoldshadows.myshopify.com,...,2022-02-25 02:37:11,NaN,NaN,Gopinath Nagalingam,NaN,Sanjay,Open,2022-02-25,NaT,2022-02-01
7,7,LE-1522,Medium,NaN,Question,TBD,Apps,Shopify Plus,Premium,naturisimo-store.myshopify.com,...,2022-02-25 02:06:54,NaN,NaN,Anshu Bhardwaj,NaN,Ranga Prakash,Open,2022-02-25,NaT,2022-02-01
8,8,LE-1521,Medium,NaN,Problem/Bug,Testing Miss,Platform,Shopify Plus,Premium,stfrock-dev.myshopify.com,...,2022-02-24 23:55:40,NaN,NaN,Anshu Bhardwaj,NaN,Sanjay,Waiting On Customer,2022-02-24,NaT,2022-02-01


## Weekly Reports

### Inflow Trend

In [32]:
import plotly.express as px

grp_date_platform = datadf.groupby(
        ['CreateDateM','Platform'],
        dropna=False
    )['Issue Key'].count().rename('Count').reset_index(name='Count').fillna("NA")

fig = px.line(grp_date_platform, x="CreateDateM", y="Count", color="Platform", title='Inflow by Month and Platform')
fig.update_traces(textposition="bottom right")

fig.update_traces(textposition="bottom right")
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor="Black",
    plot_bgcolor="Black",
    font=dict(
                color="White",
                size=12
    ),
)

fig.show()

### Created vs Resolved

In [36]:
print("Created: ", len(lastweekdfcreated))
print("Resolved: ", len(lastweekdfresolved))

Created:  14
Resolved:  21


### Group Data

In [38]:
# Group by different fields
grp_date_platform_wk = lastweekdfcreated.groupby(
        ['CreateDate','Platform'],
        dropna=False
    )['Issue Key'].count().rename('Count').reset_index(name='Count').fillna("NA")

grp_date_swymplan_wk = lastweekdfcreated.groupby(
        ['CreateDate','Swym Plan'],
        dropna=False
    )['Issue Key'].count().rename('Count').reset_index(name='Count').fillna("NA")

grp_date_platformplan_wk = lastweekdfcreated.groupby(
        ['CreateDate','Platform Plan'],
        dropna=False
    )['Issue Key'].count().rename('Count').reset_index(name='Count').fillna("NA")

grp_date_team_date = lastweekdf.groupby(
        ['CreateDate','TeamName'],
        dropna=False
    )['Issue Key'].count().rename('Count').reset_index(name='Count').fillna("NA")

# Root Cause Categories
grp_date_rca_wk = lastweekdfresolved.groupby(
        ['ResolvedDate','Root Cause'],
        dropna=False
    )['Issue Key'].count().rename('Count').reset_index(name='Count').fillna("NA")



### Created by Swym Plan

In [40]:
grp_date_swymplan_wk.groupby(['Swym Plan']).sum()

,Count
Swym Plan,
Enterprise,5
Free,1
Premium,3
Pro,3
Starter,2


### Created by Team Name

In [42]:
grp_date_team_date.groupby(['TeamName']).sum()

,Count
TeamName,
Api,1
Apps,4
BI,1
MerchantExpereicneFE,1
Platform,4
Storefront,2
Unassigned,1


### Created by Platform

In [44]:
grp_date_platformplan_wk.groupby(['Platform Plan']).sum()

,Count
Platform Plan,
Bigcommerce - Enterprise Plan Store,1
Shopify - Affiliate,2
Shopify Basic,1
Shopify Plus,8
Shopify Professional,2


### Created by Platform

In [46]:
grp_date_platform_wk.groupby(['Platform']).sum()

,Count
Platform,
BigCommerce,1
Shopify,13


### Root Causes

In [48]:
grp_date_rca_wk.groupby(['Root Cause']).sum()

,Count
Root Cause,
Code Quality,4
Design Miss,1
Documentation,7
Implementation Error,3
Known Issue,3
Requirement Miss,1
Testing Miss,2


### Root Cause JIRAs

In [52]:
from jira import JIRA, JIRAError
from collections import Counter, defaultdict
from datetime import datetime
from time import sleep

import numpy as np
import pandas as pd
import networkx as nx

url = "https://swymcorp.atlassian.net"
email = "ram.vellanki@swymcorp.com"
authtoken = "ENTER JIRA API TOKEN HERE"

jira = JIRA(options={'server': url}, basic_auth=(email, authtoken))

jql = f'project=LE AND "Category[Dropdown]" not in ("v3 Migration", "Import") and resolutiondate >= {weekstartdt} and resolutiondate < {weekenddt} and created >= startOfYear()'

jira_search = jira.search_issues(jql,fields="issuetype, created, resolutiondate, key, customfield_10079, summary, issuelinks")

rcajira = {}
for issue in jira_search:
    ilinks = issue.fields.issuelinks
    if len(ilinks) > 0:
        for rkey in ilinks:
            if rkey.type.name == 'Rootcause':
                if hasattr(rkey, 'outwardIssue'):
                    rkeval = rkey.outwardIssue.key
                elif hasattr(rkey, 'inwardIssue'):
                    rkeval = rkey.inwardIssue.key

                if rkeval in rcajira:
                    rcajira[rkeval] += 1
                else:
                    rcajira[rkeval] = 1
                    
print ("{:<8}\t {:<15}".format('JIRA KEY','COUNT'))
for k, v in rcajira.items():
    print ("{:<8}\t {:<15}".format(k, v))
    
    
# Issue is that the key is not found in dictionary causing the increment to fail

JIRA KEY	 COUNT          
STP-186 	 1              
STP-364 	 1              
STP-343 	 1              
STP-124 	 1              
STP-168 	 1              
